Intially, the reviews data that is used is the data fetched using the Yelp reviews API endpoint. This API however only returns the top 3 reviews sorted using yelp_sort for every restuarant provided the business id. Extracting information and perfoming analysis on this data is not very useful. So we proceeded to extract the reviews for every resturant in the list of our selected retaurants (as per locations and categories) from the Yelp website. This notebook has the code for extracting the Reviews for each restuarnt from the Yelp website.

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from tqdm import tqdm
import json
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
import numpy as np
import re

In [3]:
#retrieving all listed world entities
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import string
import time
import requests
from lxml import html
from bs4 import BeautifulSoup
from tqdm import tqdm
import json
import pandas as pd
import re
import json

In [4]:
from tqdm import tqdm
import time  # Import time to simulate scraping time

In [5]:
file_path = '/Users/apoorvashete/Desktop/STA220FinalProject-Yelp/restaurants_data/popular_restaurants.csv'
popular_restaurants_df = pd.read_csv(file_path)
popular_restaurants_df

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,SP4bvMvxel_UPdof_er3HA,taco-mahal-new-york,Taco Mahal,https://s3-media2.fl.yelpcdn.com/bphoto/wo7f5l...,False,https://www.yelp.com/biz/taco-mahal-new-york?a...,886,"[{'alias': 'indpak', 'title': 'Indian'}, {'ali...",4.4,"{'latitude': 40.73235768476899, 'longitude': -...","['delivery', 'pickup']",$$,"{'address1': '73 7th Ave S', 'address2': '', '...",16467191553,(646) 719-1553,3077.683720
1,16ZnHpuaaBt92XWeJHCC5A,olio-e-più-new-york-7,Olio e Più,https://s3-media4.fl.yelpcdn.com/bphoto/CUpPgz...,False,https://www.yelp.com/biz/olio-e-pi%C3%B9-new-y...,5393,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 40.733798036104304, 'longitude': ...","['delivery', 'pickup']",$$,"{'address1': '3 Greenwich Ave', 'address2': No...",12122436546,(212) 243-6546,3175.697940
2,NOh24SMS6k4pzNv8Ds7KDg,cafe-habana-new-york-4,Cafe Habana,https://s3-media3.fl.yelpcdn.com/bphoto/cUHxmL...,False,https://www.yelp.com/biz/cafe-habana-new-york-...,3311,"[{'alias': 'cuban', 'title': 'Cuban'}, {'alias...",4.0,"{'latitude': 40.72293, 'longitude': -73.99419}","['pickup', 'delivery']",$$,"{'address1': '17 Prince St', 'address2': '', '...",12126252001,(212) 625-2001,1933.430593
3,0CjK3esfpFcxIopebzjFxA,joes-shanghai-new-york-2,Joe's Shanghai,https://s3-media2.fl.yelpcdn.com/bphoto/ORdfWy...,False,https://www.yelp.com/biz/joes-shanghai-new-yor...,7327,"[{'alias': 'shanghainese', 'title': 'Shanghain...",3.8,"{'latitude': 40.7156608, 'longitude': -73.9967...","['pickup', 'delivery']",$$,"{'address1': '46 Bowery St', 'address2': '', '...",12122338888,(212) 233-8888,1143.405792
4,FlZ1zdVEKWv7dwqm8Uw8-w,raku-new-york-7,Raku,https://s3-media4.fl.yelpcdn.com/bphoto/GEbZnH...,False,https://www.yelp.com/biz/raku-new-york-7?adjus...,1635,"[{'alias': 'japanese', 'title': 'Japanese'}, {...",4.5,"{'latitude': 40.7264988089246, 'longitude': -7...","['pickup', 'delivery']",$$,"{'address1': '342 E 6th St', 'address2': '', '...",12122281324,(212) 228-1324,2416.602715
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,NL2gOp0p7D3_YNI3eZfbPw,india-grill-arlington-2,India Grill,https://s3-media3.fl.yelpcdn.com/bphoto/4pRlXd...,False,https://www.yelp.com/biz/india-grill-arlington...,622,"[{'alias': 'indpak', 'title': 'Indian'}]",4.2,"{'latitude': 32.68123, 'longitude': -97.11371}","['pickup', 'delivery']",$$,"{'address1': '3900 Chaney Dr', 'address2': 'St...",18174689150,(817) 468-9150,3294.696270
251,v6xe2MRwStuwxPH2SH89ag,italian-cafe-irving,Italian Cafe,https://s3-media2.fl.yelpcdn.com/bphoto/c9_hmX...,False,https://www.yelp.com/biz/italian-cafe-irving?a...,470,"[{'alias': 'italian', 'title': 'Italian'}]",3.9,"{'latitude': 32.865059007048, 'longitude': -96...","['pickup', 'delivery']",$$,"{'address1': '387 E Las Colinas Blvd', 'addres...",19724010216,(972) 401-0216,24275.084419
252,QtCs9kugi0cs4x_O1QDaPg,hugos-invitados-irving,Hugo's Invitados,https://s3-media1.fl.yelpcdn.com/bphoto/uRsSLs...,False,https://www.yelp.com/biz/hugos-invitados-irvin...,929,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...",4.3,"{'latitude': 32.869753, 'longitude': -96.937645}","['pickup', 'delivery']",$$$,"{'address1': ""5240 N O'Connor Blvd"", 'address2...",12144960590,(214) 496-0590,24574.172420
253,QXOIJVTUcXPN_2LOvevSxg,bethany-boba-tea-and-cafe-arlington,Bethany Boba Tea & Cafe,https://s3-media1.fl.yelpcdn.com/bphoto/Rpip7o...,False,https://www.yelp.com/biz/bethany-boba-tea-and-...,719,"[{'alias': 'chinese', 'title': 'Chinese'}, {'a...",3.9,"{'latitude': 32.7213249, 'longitude': -97.1155...",['delivery'],$,"{'address1': '705 W Park Row Dr', 'address2': ...",18174611245,(817) 461-1245,1330.808818


In [18]:
#extracting links and corresponding business ids for every restaurant
links_and_ids = popular_restaurants_df[['url', 'id']]
links_and_ids

,url,id
0,https://www.yelp.com/biz/taco-mahal-new-york?a...,SP4bvMvxel_UPdof_er3HA
1,https://www.yelp.com/biz/olio-e-pi%C3%B9-new-y...,16ZnHpuaaBt92XWeJHCC5A
2,https://www.yelp.com/biz/cafe-habana-new-york-...,NOh24SMS6k4pzNv8Ds7KDg
3,https://www.yelp.com/biz/joes-shanghai-new-yor...,0CjK3esfpFcxIopebzjFxA
4,https://www.yelp.com/biz/raku-new-york-7?adjus...,FlZ1zdVEKWv7dwqm8Uw8-w
...,...,...
250,https://www.yelp.com/biz/india-grill-arlington...,NL2gOp0p7D3_YNI3eZfbPw
251,https://www.yelp.com/biz/italian-cafe-irving?a...,v6xe2MRwStuwxPH2SH89ag
252,https://www.yelp.com/biz/hugos-invitados-irvin...,QtCs9kugi0cs4x_O1QDaPg
253,https://www.yelp.com/biz/bethany-boba-tea-and-...,QXOIJVTUcXPN_2LOvevSxg


In [8]:
#extracting links of all restaurants
restaurant_urls = popular_restaurants_df['url'].tolist()
restaurant_urls

['https://www.yelp.com/biz/taco-mahal-new-york?adjust_creative=mUhtpg0LANTRd76OAeN1RQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=mUhtpg0LANTRd76OAeN1RQ',
 'https://www.yelp.com/biz/olio-e-pi%C3%B9-new-york-7?adjust_creative=mUhtpg0LANTRd76OAeN1RQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=mUhtpg0LANTRd76OAeN1RQ',
 'https://www.yelp.com/biz/cafe-habana-new-york-4?adjust_creative=mUhtpg0LANTRd76OAeN1RQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=mUhtpg0LANTRd76OAeN1RQ',
 'https://www.yelp.com/biz/joes-shanghai-new-york-2?adjust_creative=mUhtpg0LANTRd76OAeN1RQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=mUhtpg0LANTRd76OAeN1RQ',
 'https://www.yelp.com/biz/raku-new-york-7?adjust_creative=mUhtpg0LANTRd76OAeN1RQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=mUhtpg0LANTRd76OAeN1RQ',
 'https://www.yelp.com/biz/himalayan-yak-jackson-heights?adjust_creative=mUhtpg0LANT

In [22]:
# Function to scrape data from a given page URL
def scrape_page(url, business_id):
    result = []

    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    li_class = soup.find_all('li', class_='css-1q2nwpv')

    for li in li_class:
        # Initialize variables
        date_text = ''
        review_text = ''
        rating = ''
        
        # Date
        date_span = li.find('span', class_='css-chan6m')
        if date_span:
            date_text = date_span.text.strip()

        # Review text
        review_text_span = li.find('span', class_='raw__09f24__T4Ezm')
        if review_text_span:
            review_text = review_text_span.text.strip()

        # Rating
        rating_div = li.find('div', class_='css-14g69b3')
        if rating_div:
            rating = rating_div.get('aria-label')

        # Check if all three elements are found before appending to the result list
        if date_text and review_text and rating:
            result.append({
                'Date': date_text,
                'Review Text': review_text,
                'Rating': rating, 
                'Business ID': business_id
            })

    return result

In [23]:
# Function to navigate to the next page
def navigate_to_next_page(driver):
    # button for going to the next page
    #navigation-button-container__09f24__SvcBh css-1qn0b6x--> next page button class 
    try:
        button = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "/html/body/yelp-react-root/div[1]/div[6]/div/div[1]/div[1]/main/div[3]/div/section/div[2]/div[5]/div[1]/div/div[11]"))
        )
        button.click()
    except TimeoutException:
        print("Timed out waiting for the button to be present.")

In [30]:
# Install or update ChromeDriver
ChromeDriverManager().install()
driver = webdriver.Chrome()

In [17]:
# #trying fucntions for the first link
# # Main code
# url= restaurant_urls[0] # Replace with your actual base URL
# num_pages = 5
# result_for_link_1 = []

# #initialise the driver
# driver.set_page_load_timeout(20)
# try:
#     driver.get(url)
# except TimeoutException:
#     driver.execute_script("window.stop();")

# for page_number in range(1, 6):
#     scrape_result = scrape_page(url)
#     print(f"Scraped data from page {page_number}")
#     result_for_link_1.extend(scrape_result)
#     # Navigate to the next page
#     navigate_to_next_page(driver)
#     url = driver.current_url

# # Quit the WebDriver when done
# driver.quit()

Scraped data from page 1
Scraped data from page 2
Scraped data from page 3
Scraped data from page 4
Scraped data from page 5


In [13]:
# Loop through each restaurant URL
all_data = []
for url in tqdm(restaurant_urls, desc="Scraping Restaurants"):
    num_pages = 5

    # Initialize the driver
    driver.set_page_load_timeout(20)
    try:
        driver.get(url)
    except TimeoutException:
        driver.execute_script("window.stop();")

    # Loop through each page for the current restaurant URL
    for page_number in range(1, num_pages + 1):
        scrape_result = scrape_page(url)
        print(f"Scraped data from page {page_number}")
        time.sleep(1)  # Simulate scraping time
        all_data.extend(scrape_result)

        # Navigate to the next page
        navigate_to_next_page(driver)
        url = driver.current_url

    print(f"\nResults fetched for URL {url}:")

# Quit the driver after scraping all URLs
driver.quit()

Scraping Restaurants:   0%|          | 0/255 [00:00<?, ?it/s]

Scraped data from page 1
Scraped data from page 2
Scraped data from page 3
Scraped data from page 4
Scraped data from page 5


Scraping Restaurants:   0%|          | 1/255 [00:29<2:06:53, 29.97s/it]


Results fetched for URL https://www.yelp.com/biz/taco-mahal-new-york?adjust_creative=mUhtpg0LANTRd76OAeN1RQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=mUhtpg0LANTRd76OAeN1RQ&start=40:
Scraped data from page 1
Scraped data from page 2
Scraped data from page 3
Scraped data from page 4
Scraped data from page 5


Scraping Restaurants:   1%|          | 2/255 [00:54<1:53:03, 26.81s/it]

...


Results fetched for URL https://www.yelp.com/biz/hugos-invitados-irving?adjust_creative=mUhtpg0LANTRd76OAeN1RQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=mUhtpg0LANTRd76OAeN1RQ&start=40:
Scraped data from page 1
Scraped data from page 2
Scraped data from page 3
Scraped data from page 4
Scraped data from page 5


Scraping Restaurants: 100%|█████████▉| 254/255 [1:54:55<00:26, 26.13s/it]


Results fetched for URL https://www.yelp.com/biz/bethany-boba-tea-and-cafe-arlington?adjust_creative=mUhtpg0LANTRd76OAeN1RQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=mUhtpg0LANTRd76OAeN1RQ&start=40:
Scraped data from page 1
Scraped data from page 2
Scraped data from page 3
Scraped data from page 4
Scraped data from page 5


Scraping Restaurants: 100%|██████████| 255/255 [1:55:22<00:00, 27.15s/it]


Results fetched for URL https://www.yelp.com/biz/mr-max-irving?adjust_creative=mUhtpg0LANTRd76OAeN1RQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=mUhtpg0LANTRd76OAeN1RQ&start=40:


In [31]:
# Loop through each row in the DataFrame
all_data = []
for index, row in tqdm(links_and_ids.iterrows(), total=len(links_and_ids), desc="Scraping Restaurants"):
    url = row['url']
    business_id = row['id']

    # Initialize the driver
    driver.set_page_load_timeout(20)
    try:
        driver.get(url)
    except TimeoutException:
        driver.execute_script("window.stop();")


    num_pages = 5

    # Loop through each page for the current restaurant URL
    for page_number in range(1, num_pages + 1):
        scrape_result = scrape_page(url, business_id)
        print(f"Scraped data from page {page_number}")
        time.sleep(1)  # Simulate scraping time
        all_data.extend(scrape_result)

        # Navigate to the next page
        navigate_to_next_page(driver)
        url = driver.current_url

    print(f"\nResults fetched for URL {url}:")

# Quit the driver after scraping all URLs
driver.quit()

Scraping Restaurants:   0%|          | 0/255 [00:00<?, ?it/s]

Scraped data from page 1
Scraped data from page 2
Scraped data from page 3
Scraped data from page 4
Scraped data from page 5


Scraping Restaurants:   0%|          | 1/255 [00:28<2:00:27, 28.46s/it]


Results fetched for URL https://www.yelp.com/biz/taco-mahal-new-york?adjust_creative=mUhtpg0LANTRd76OAeN1RQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=mUhtpg0LANTRd76OAeN1RQ&start=40:
Scraped data from page 1
Scraped data from page 2
Scraped data from page 3
Scraped data from page 4
Scraped data from page 5


Scraping Restaurants:   1%|          | 2/255 [00:58<2:04:12, 29.46s/it]

...


Results fetched for URL https://www.yelp.com/biz/hugos-invitados-irving?adjust_creative=mUhtpg0LANTRd76OAeN1RQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=mUhtpg0LANTRd76OAeN1RQ&start=40:
Scraped data from page 1
Scraped data from page 2
Scraped data from page 3
Scraped data from page 4
Scraped data from page 5


Scraping Restaurants: 100%|█████████▉| 254/255 [2:11:18<00:29, 29.26s/it]


Results fetched for URL https://www.yelp.com/biz/bethany-boba-tea-and-cafe-arlington?adjust_creative=mUhtpg0LANTRd76OAeN1RQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=mUhtpg0LANTRd76OAeN1RQ&start=40:
Scraped data from page 1
Scraped data from page 2
Scraped data from page 3
Scraped data from page 4
Scraped data from page 5


Scraping Restaurants: 100%|██████████| 255/255 [2:11:49<00:00, 31.02s/it]


Results fetched for URL https://www.yelp.com/biz/mr-max-irving?adjust_creative=mUhtpg0LANTRd76OAeN1RQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=mUhtpg0LANTRd76OAeN1RQ&start=40:


In [29]:
driver.quit()

Code for Mulithreading

In [1]:
# # Function to scrape a single page
# def scrape_single_page(url, driver):
#     scrape_result = scrape_page(driver, url)
#     time.sleep(1)  # Simulate scraping time
#     return scrape_result

# # Function to scrape all pages for a restaurant URL
# def scrape_all_pages(url, num_pages):
#     all_data_ = []
#     # Install or update ChromeDriver
#     ChromeDriverManager().install()
#     # Now you can use the webdriver
#     local_driver = webdriver.Chrome()
#     # local_driver = webdriver.Chrome(executable_path='/path/to/chromedriver')  # Replace with your actual path

#     local_driver.set_page_load_timeout(20)

#     try:
#         local_driver.get(url)
#     except TimeoutException:
#         local_driver.execute_script("window.stop();")

#     for page_number in range(1, num_pages + 1):
#         scrape_result = scrape_single_page(url, local_driver)
#         print(f"Scraped data from page {page_number}")
#         all_data_.extend(scrape_result)

#         # Navigate to the next page
#         navigate_to_next_page(local_driver)
#         url = local_driver.current_url

#     print(f"\nResults fetched for URL {url}:")

#     # Quit the local driver before exiting the function
#     local_driver.quit()

#     return all_data_

# # Loop through each restaurant URL concurrently
# all_data__ = []
# num_pages = 5

# with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
#     futures = [executor.submit(scrape_all_pages, url, num_pages) for url in restaurant_urls]

#     # tqdm is used to display a progress bar
#     for future in tqdm(concurrent.futures.as_completed(futures), total=len(restaurant_urls), desc="Scraping Restaurants"):
#         try:
#             result = future.result()
#             all_data__.extend(result)
#         except Exception as e:
#             print(f"Error: {e}")

# # No need to quit the driver here since it's already done in each thread

In [14]:
# import concurrent.futures
# from tqdm import tqdm

# # Function to scrape a single page
# def scrape_single_page(url):
#     scrape_result = scrape_page(url)
#     time.sleep(1)  # Simulate scraping time
#     return scrape_result

# # Function to scrape all pages for a restaurant URL
# def scrape_all_pages(url, num_pages):
#     all_data_ = []
#     driver.set_page_load_timeout(20)

#     try:
#         driver.get(url)
#     except TimeoutException:
#         driver.execute_script("window.stop();")

#     for page_number in range(1, num_pages + 1):
#         scrape_result = scrape_single_page(url)
#         print(f"Scraped data from page {page_number}")
#         all_data_.extend(scrape_result)

#         # Navigate to the next page
#         navigate_to_next_page(driver)
#         url = driver.current_url

#     print(f"\nResults fetched for URL {url}:")
#     return all_data_

# # Loop through each restaurant URL concurrently
# all_data__ = []
# num_pages = 5

# with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
#     futures = [executor.submit(scrape_all_pages, url, num_pages) for url in restaurant_urls]

#     # tqdm is used to display a progress bar
#     for future in tqdm(concurrent.futures.as_completed(futures), total=len(restaurant_urls), desc="Scraping Restaurants"):
#         try:
#             result = future.result()
#             all_data__.extend(result)
#         except Exception as e:
#             print(f"Error: {e}")

# # Quit the driver after scraping all URLs
# driver.quit()

This is the data for reviews of restaurants without the business_ids appended for every review 

In [14]:
# Write the data to the JSON file

json_file_path = 'reviews_data.json'

with open(json_file_path, 'w', encoding='utf-8') as json_file:
    json.dump(all_data, json_file, ensure_ascii=False, indent=2)

print(f'Data has been successfully written to {json_file_path}')

Data has been successfully written to reviews_data.json


In [15]:
json_file_path = 'reviews_data.json'

# Read data from the JSON file
with open(json_file_path, 'r', encoding='utf-8') as json_file:
    data = json.load(json_file)

In [16]:
df = pd.DataFrame(data)

In [17]:
df

,Date,Review Text,Rating
0,"Mar 3, 2024",Taco Mahal was a very unique experience. I've ...,5 star rating
1,"Feb 19, 2024",1/1 Ambience; 1.5/2 Food + Beverages; 1/1 Serv...,5 star rating
2,"Feb 4, 2024",delicious indian food and fast!! we ordered:- ...,5 star rating
3,"Jan 28, 2024",For those that know my reviews if there is any...,4 star rating
4,"Mar 10, 2024","We stopped by for a ""life-changing taco"" as pa...",5 star rating
...,...,...,...
11965,"Jun 14, 2023","Once you step into Mr Max, it feels as if you ...",5 star rating
11966,"Jan 21, 2023",The restaurant setting is amazing. There is ei...,5 star rating
11967,"Jan 26, 2024",Best restaurant in all of Dallas! I've been he...,5 star rating
11968,"Dec 17, 2023",Limited seatings a total hole in a wall type o...,5 star rating


This is the data for reviews of restaurants with the business_ids appended for every review 

In [32]:

json_file_path = 'final_reviews_data.json'

# Write the data to the JSON file
with open(json_file_path, 'w', encoding='utf-8') as json_file:
    json.dump(all_data, json_file, ensure_ascii=False, indent=2)

print(f'Data has been successfully written to {json_file_path}')

Data has been successfully written to final_reviews_data.json


In [33]:
json_file_path = 'final_reviews_data.json'

# Read data from the JSON file
with open(json_file_path, 'r', encoding='utf-8') as json_file:
    data = json.load(json_file)

In [34]:
df = pd.DataFrame(data)

In [35]:
df

,Date,Review Text,Rating,Business ID
0,"Mar 3, 2024",Taco Mahal was a very unique experience. I've ...,5 star rating,SP4bvMvxel_UPdof_er3HA
1,"Feb 19, 2024",1/1 Ambience; 1.5/2 Food + Beverages; 1/1 Serv...,5 star rating,SP4bvMvxel_UPdof_er3HA
2,"Feb 4, 2024",delicious indian food and fast!! we ordered:- ...,5 star rating,SP4bvMvxel_UPdof_er3HA
3,"Mar 10, 2024","We stopped by for a ""life-changing taco"" as pa...",5 star rating,SP4bvMvxel_UPdof_er3HA
4,"Jan 28, 2024",For those that know my reviews if there is any...,4 star rating,SP4bvMvxel_UPdof_er3HA
...,...,...,...,...
11945,"Jun 14, 2023","Once you step into Mr Max, it feels as if you ...",5 star rating,P0VziPp_DuKBol_YNKLqSw
11946,"Jan 21, 2023",The restaurant setting is amazing. There is ei...,5 star rating,P0VziPp_DuKBol_YNKLqSw
11947,"Jan 26, 2024",Best restaurant in all of Dallas! I've been he...,5 star rating,P0VziPp_DuKBol_YNKLqSw
11948,"Dec 17, 2023",Limited seatings a total hole in a wall type o...,5 star rating,P0VziPp_DuKBol_YNKLqSw
